In [30]:
#*--------------*
#| Dependencies |
#*--------------*
import os
from pyspark.sql import SparkSession
import time
import findspark
from pyspark import SparkFiles

# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.2'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 222 kB in 1s (157 kB/s)
Reading package lists... Done


In [4]:
#creating a Spark Session
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [7]:
#reading in home_sales data from the AWS S3 bucket.
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
homes_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True, quote="\"", escape="\"")
homes_df.show(5)


+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16 00:00:00|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08 00:00:00|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+-------

In [9]:
# creating a temporary view of the df.
homes_df.createOrReplaceTempView("homes")


In [10]:
homes_df.schema

StructType([StructField('id', StringType(), True), StructField('date', TimestampType(), True), StructField('date_built', IntegerType(), True), StructField('price', IntegerType(), True), StructField('bedrooms', IntegerType(), True), StructField('bathrooms', IntegerType(), True), StructField('sqft_living', IntegerType(), True), StructField('sqft_lot', IntegerType(), True), StructField('floors', IntegerType(), True), StructField('waterfront', IntegerType(), True), StructField('view', IntegerType(), True)])

In [11]:
#What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""SELECT YEAR(date) as year_sold, ROUND(AVG(price),2) as 4bd_avg_price
FROM homes
WHERE bedrooms = 4
GROUP BY year_sold
ORDER BY year_sold ASC
""").show()


+---------+-------------+
|year_sold|4bd_avg_price|
+---------+-------------+
|     2019|     300263.7|
|     2020|    298353.78|
|     2021|    301819.44|
|     2022|    296363.88|
+---------+-------------+



In [12]:
#What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""SELECT date_built, ROUND(AVG(price),2) as 3bd3b_avg_price
FROM homes
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built ASC""").show()


+----------+---------------+
|date_built|3bd3b_avg_price|
+----------+---------------+
|      2010|      292859.62|
|      2011|      291117.47|
|      2012|      293683.19|
|      2013|      295962.27|
|      2014|      290852.27|
|      2015|       288770.3|
|      2016|      290555.07|
|      2017|      292676.79|
+----------+---------------+



In [13]:
# What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""SELECT date_built, ROUND(AVG(price),2) as 3bd3b_avg_price, ROUND(AVG(sqft_living),2) as avg_sqft
FROM homes
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built ASC""").show()


+----------+---------------+--------+
|date_built|3bd3b_avg_price|avg_sqft|
+----------+---------------+--------+
|      2010|      285010.22| 2264.46|
|      2011|      276553.81|  2264.1|
|      2012|      307539.97| 2334.57|
|      2013|      303676.79| 2296.37|
|      2014|      298264.72|  2285.6|
|      2015|      297609.97| 2308.93|
|      2016|       293965.1| 2274.17|
|      2017|      280317.58| 2274.01|
+----------+---------------+--------+



In [15]:
# What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

sql = """SELECT view, ROUND(AVG(price),2) as avg_price
FROM homes 
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""
spark.sql(sql).show(10)

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
+----+----------+
only showing top 10 rows

--- 1.0891053676605225 seconds ---


In [16]:
# Cache the the temporary table home_sales.
spark.sql('cache table homes')

DataFrame[]

In [18]:
# Check if the table is cached.
spark.catalog.isCached('homes')

True

In [19]:
# Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql(sql).show(10)

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
+----+----------+
only showing top 10 rows

--- 0.5521900653839111 seconds ---


In [20]:
# Partition by the "date_built" field on the formatted parquet home sales data 
homes_df.write.partitionBy('date_built').parquet('p_homes',mode='overwrite') 


In [21]:
# Read the parquet formatted data.
p_homes = spark.read.parquet("p_homes")

In [23]:
# Create a temporary table for the parquet data.
p_homes.createOrReplaceTempView("p_homes")

In [24]:
# Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql(sql).show(10)

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
+----+----------+
only showing top 10 rows

--- 0.4563748836517334 seconds ---


In [25]:
# Uncache the home_sales temporary table.
spark.sql("uncache table homes")


DataFrame[]

In [26]:
# Check if the home_sales is no longer cached
spark.catalog.isCached('homes')

False